# OmniQuant sweep summary (let_lr × lwc_lr)

This notebook scans a run directory (e.g. `./log/opt-1.3b/`), reads each `results.json`, and builds tables keyed by `(let_lr, lwc_lr)`.

It is robust to:
- `let_lr` / `lwc_lr` stored inside `results.json`, **or**
- extracting them from the folder name like `letlr_1e-02_lwclr_1e-03`.


In [3]:
from pathlib import Path
import json
import re
import pandas as pd
import numpy as np

# -------------------------------------------------------------------
# Configure this path (relative to where you launch" Jupyter).
# Example: Path("./log/opt-1.3b")
# -------------------------------------------------------------------
BASE_DIR = Path("./log/opt-1.3b")

# If your runs are nested deeper, you can widen the glob below.
GLOB_PATTERN = "**/results.json"
print("Scanning:", BASE_DIR.resolve())


Scanning: /workspace/orkspace/OmniQuant-main/log/opt-1.3b


In [4]:
def _safe_float(x):
    try:
        return float(x)
    except Exception:
        return np.nan

def _parse_lr_from_run_name(run_name: str):
    # accepts: letlr_1e-02_lwclr_1e-03  (or similar)
    m1 = re.search(r"letlr_([0-9.]+e[+-]?[0-9]+)", run_name)
    m2 = re.search(r"lwclr_([0-9.]+e[+-]?[0-9]+)", run_name)
    let_lr = _safe_float(m1.group(1)) if m1 else np.nan
    lwc_lr = _safe_float(m2.group(1)) if m2 else np.nan
    return let_lr, lwc_lr

def _extract_ppl(results_obj: dict):
    # Expect something like:
    # results_obj["ppl"]["wikitext:wikitext-103-raw-v1::test"]["ppl"]
    ppl_block = results_obj.get("ppl", {})
    def find_dataset_ppl(substr: str):
        if not isinstance(ppl_block, dict):
            return np.nan
        # Direct match by key scan
        for k, v in ppl_block.items():
            if substr in k and isinstance(v, dict):
                if "ppl" in v:
                    return _safe_float(v["ppl"])
        return np.nan

    wt = find_dataset_ppl("wikitext")
    lb = find_dataset_ppl("lambada")
    return wt, lb

def load_runs(base_dir: Path, glob_pattern: str = "**/results.json"):
    rows = []
    for p in sorted(base_dir.glob(glob_pattern)):
        try:
            obj = json.loads(p.read_text())
        except Exception as e:
            rows.append({
                "results_path": str(p),
                "run_dir": str(p.parent),
                "let_lr": np.nan,
                "lwc_lr": np.nan,
                "wikitext_ppl": np.nan,
                "lambada_ppl": np.nan,
                "error": f"json_load_error: {e}",
            })
            continue

        run_dir = obj.get("run_dir", str(p.parent))
        let_lr = obj.get("let_lr", np.nan)
        lwc_lr = obj.get("lwc_lr", np.nan)

        # fallback to parsing from folder name if missing
        if (let_lr is None) or (not np.isfinite(_safe_float(let_lr))):
            pl, _ = _parse_lr_from_run_name(Path(run_dir).name)
            let_lr = pl
        else:
            let_lr = _safe_float(let_lr)

        if (lwc_lr is None) or (not np.isfinite(_safe_float(lwc_lr))):
            _, pl = _parse_lr_from_run_name(Path(run_dir).name)
            lwc_lr = pl
        else:
            lwc_lr = _safe_float(lwc_lr)

        wt, lb = _extract_ppl(obj)

        rows.append({
            "results_path": str(p),
            "run_dir": str(run_dir),
            "run_name": Path(run_dir).name,
            "let_lr": let_lr,
            "lwc_lr": lwc_lr,
            "wikitext_ppl": wt,
            "lambada_ppl": lb,
            "error": "",
        })

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    # Normalize scientific formatting columns for readability
    df["let_lr_str"] = df["let_lr"].map(lambda x: f"{x:.0e}" if np.isfinite(x) else "NA")
    df["lwc_lr_str"] = df["lwc_lr"].map(lambda x: f"{x:.0e}" if np.isfinite(x) else "NA")
    return df

df = load_runs(BASE_DIR, GLOB_PATTERN)
df.head()


,results_path,run_dir,run_name,let_lr,lwc_lr,wikitext_ppl,lambada_ppl,error,let_lr_str,lwc_lr_str
0,log/opt-1.3b/letlr_1e-02_lwclr_1e-02/results.json,log/opt-1.3b/letlr_1e-02_lwclr_1e-02,letlr_1e-02_lwclr_1e-02,0.010,0.0100,10810.035499,10057.069626,,1e-02,1e-02
1,log/opt-1.3b/letlr_1e-02_lwclr_1e-03/results.json,log/opt-1.3b/letlr_1e-02_lwclr_1e-03,letlr_1e-02_lwclr_1e-03,0.010,0.0010,10810.035499,10057.069626,,1e-02,1e-03
2,log/opt-1.3b/letlr_1e-02_lwclr_1e-04/results.json,log/opt-1.3b/letlr_1e-02_lwclr_1e-04,letlr_1e-02_lwclr_1e-04,0.010,0.0001,10810.035499,10057.069626,,1e-02,1e-04
3,log/opt-1.3b/letlr_1e-03_lwclr_1e-02/results.json,log/opt-1.3b/letlr_1e-03_lwclr_1e-02,letlr_1e-03_lwclr_1e-02,0.001,0.0100,2669.253399,1668.704285,,1e-03,1e-02
4,log/opt-1.3b/letlr_1e-03_lwclr_1e-03/results.json,log/opt-1.3b/letlr_1e-03_lwclr_1e-03,letlr_1e-03_lwclr_1e-03,0.001,0.0010,2669.253399,1668.704285,,1e-03,1e-03


In [5]:
if df.empty:
    print("No results.json files found under", BASE_DIR)
else:
    print(f"Found {len(df)} runs")
    # show any parse errors
    bad = df[df["error"] != ""]
    if len(bad):
        print(f"Warnings: {len(bad)} runs had errors (showing first 5):")
        display(bad.head(5)[["results_path","error"]])


Found 9 runs


## Long-form table (one row per run)

In [6]:
if not df.empty:
    # Sort by wikitext ppl (ascending)
    df_sorted = df.sort_values(["wikitext_ppl", "lambada_ppl"], ascending=True, na_position="last").reset_index(drop=True)
    display(df_sorted[[
        "let_lr_str","lwc_lr_str","wikitext_ppl","lambada_ppl","run_dir"
    ]])


,let_lr_str,lwc_lr_str,wikitext_ppl,lambada_ppl,run_dir
0,1e-04,1e-02,28.802450,33.222303,log/opt-1.3b/letlr_1e-04_lwclr_1e-02
1,1e-04,1e-03,28.802450,33.222303,log/opt-1.3b/letlr_1e-04_lwclr_1e-03
2,1e-04,1e-04,28.802450,33.222303,log/opt-1.3b/letlr_1e-04_lwclr_1e-04
3,1e-03,1e-02,2669.253399,1668.704285,log/opt-1.3b/letlr_1e-03_lwclr_1e-02
4,1e-03,1e-03,2669.253399,1668.704285,log/opt-1.3b/letlr_1e-03_lwclr_1e-03
5,1e-03,1e-04,2669.253399,1668.704285,log/opt-1.3b/letlr_1e-03_lwclr_1e-04
6,1e-02,1e-02,10810.035499,10057.069626,log/opt-1.3b/letlr_1e-02_lwclr_1e-02
7,1e-02,1e-03,10810.035499,10057.069626,log/opt-1.3b/letlr_1e-02_lwclr_1e-03
8,1e-02,1e-04,10810.035499,10057.069626,log/opt-1.3b/letlr_1e-02_lwclr_1e-04


## Pivot tables (let_lr × lwc_lr)

In [7]:
if not df.empty:
    pivot_wt = df.pivot_table(index="let_lr_str", columns="lwc_lr_str", values="wikitext_ppl", aggfunc="min")
    pivot_lb = df.pivot_table(index="let_lr_str", columns="lwc_lr_str", values="lambada_ppl", aggfunc="min")
    print("Wikitext PPL")
    display(pivot_wt)
    print("Lambada PPL")
    display(pivot_lb)


Wikitext PPL


lwc_lr_str,1e-02,1e-03,1e-04
let_lr_str,,,
1e-02,10810.035499,10810.035499,10810.035499
1e-03,2669.253399,2669.253399,2669.253399
1e-04,28.802450,28.802450,28.802450


Lambada PPL


lwc_lr_str,1e-02,1e-03,1e-04
let_lr_str,,,
1e-02,10057.069626,10057.069626,10057.069626
1e-03,1668.704285,1668.704285,1668.704285
1e-04,33.222303,33.222303,33.222303


## Best setting(s)

In [8]:
if not df.empty:
    best = df.sort_values(["wikitext_ppl", "lambada_ppl"], ascending=True, na_position="last").head(10)
    display(best[["let_lr_str","lwc_lr_str","wikitext_ppl","lambada_ppl","run_dir","results_path"]])


,let_lr_str,lwc_lr_str,wikitext_ppl,lambada_ppl,run_dir,results_path
6,1e-04,1e-02,28.802450,33.222303,log/opt-1.3b/letlr_1e-04_lwclr_1e-02,log/opt-1.3b/letlr_1e-04_lwclr_1e-02/results.json
7,1e-04,1e-03,28.802450,33.222303,log/opt-1.3b/letlr_1e-04_lwclr_1e-03,log/opt-1.3b/letlr_1e-04_lwclr_1e-03/results.json
8,1e-04,1e-04,28.802450,33.222303,log/opt-1.3b/letlr_1e-04_lwclr_1e-04,log/opt-1.3b/letlr_1e-04_lwclr_1e-04/results.json
3,1e-03,1e-02,2669.253399,1668.704285,log/opt-1.3b/letlr_1e-03_lwclr_1e-02,log/opt-1.3b/letlr_1e-03_lwclr_1e-02/results.json
4,1e-03,1e-03,2669.253399,1668.704285,log/opt-1.3b/letlr_1e-03_lwclr_1e-03,log/opt-1.3b/letlr_1e-03_lwclr_1e-03/results.json
5,1e-03,1e-04,2669.253399,1668.704285,log/opt-1.3b/letlr_1e-03_lwclr_1e-04,log/opt-1.3b/letlr_1e-03_lwclr_1e-04/results.json
0,1e-02,1e-02,10810.035499,10057.069626,log/opt-1.3b/letlr_1e-02_lwclr_1e-02,log/opt-1.3b/letlr_1e-02_lwclr_1e-02/results.json
1,1e-02,1e-03,10810.035499,10057.069626,log/opt-1.3b/letlr_1e-02_lwclr_1e-03,log/opt-1.3b/letlr_1e-02_lwclr_1e-03/results.json
2,1e-02,1e-04,10810.035499,10057.069626,log/opt-1.3b/letlr_1e-02_lwclr_1e-04,log/opt-1.3b/letlr_1e-02_lwclr_1e-04/results.json


## Optional: export CSVs

In [9]:
OUT_DIR = BASE_DIR
if not df.empty:
    long_csv = OUT_DIR / "sweep_table_long.csv"
    wt_csv = OUT_DIR / "sweep_table_wikitext_pivot.csv"
    lb_csv = OUT_DIR / "sweep_table_lambada_pivot.csv"

    df.sort_values(["let_lr","lwc_lr"]).to_csv(long_csv, index=False)
    df.pivot_table(index="let_lr_str", columns="lwc_lr_str", values="wikitext_ppl", aggfunc="min").to_csv(wt_csv)
    df.pivot_table(index="let_lr_str", columns="lwc_lr_str", values="lambada_ppl", aggfunc="min").to_csv(lb_csv)

    print("Wrote:")
    print(" -", long_csv)
    print(" -", wt_csv)
    print(" -", lb_csv)


Wrote:
 - log/opt-1.3b/sweep_table_long.csv
 - log/opt-1.3b/sweep_table_wikitext_pivot.csv
 - log/opt-1.3b/sweep_table_lambada_pivot.csv
